In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%writefile .env

TAVILY_API_KEY = your_key
JINA_API_KEY = your_key
CHROME_INSTANCE_PATH = /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
BROWSER_HEADLESS=False


Overwriting .env


In [3]:
import sys
from src.workflow import run_agent_workflow

In [4]:
user_query = '''
    I would like to analyze Amazon product sales data.
    The target for analysis is the './data/Amazon_Sale_Report.csv' file.
    Please conduct an analysis to extract marketing insights based on this data.
    Please perform various analytical techniques starting from basic data attribute exploration, product sales trends, variable relationships, variable combinations, etc.
    If there are any additional analyses needed to extract insights after the data analysis, please perform those as well.
    Please include detailed analysis in the report along with supporting images and charts.
    Please save the final report in PDF format.
'''

user_query = '''
    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.
    분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다.
    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.
    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.
    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.
    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.
    최종 리포트는 pdf 형태로 저장해 주세요.
'''

#user_query = "안녕 나는 장동진이야"
#user_query = '''
#    이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 간단하게 1개정도만 해 주세요. 코드 테스트 중이거든요 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요.
#    '''

# user_query = '''
#     영문으로 작성된 pptx 파일을 한글로 변환하고 싶습니다.
#     대상은 "./data/bedrock.pptx" 파일 입니다.
#     슬라이드의 영문을 한글로 번역하고 슬라이드 structure는 유지해 주세요.
# '''

# user_query = '''
#     주어진 프로젝트 소스코드는 "Bedrock-Manus: AI automation framework optimized for Amazon Bedrock and business use cases 입니다."
#     프로젝트 소스코드는 "./data/08_bedrock_manus.tar.gz" 파일 입니다.
#     나는 Streamlit을 이용해서 이 소스코드에 대한 UI를 만들고 싶습니다. 
#     제가 원하는 요구사항은
#     1. 현재 스텝이 무엇인지 (coodinator, supervisor, planner 등등) 보이는 곳이 있으면 좋겠습니다.
#         - 스텝에 대한 자세한 정보는 "./README.md"를 참고하시면 됩니다. 
#     2. 각 스텝에서 진행되는 것이 보였으면 좋겠습니다. 즉, 각 스텝에서 llm이 생성하는 토큰들을 실시간으로 볼 수 있으면 좋겠습니다. 
#     3. "app.py" 파일만 생성하지 말고, 필요하다면 기존의 다른 파일 (workflow.py 등)들도  수정하세요. 
#     4. 수정된 파일은 모두 ./artifacts 에 넣어주세요.
#     참고할 사항이 있습니다. 
#     1. 소스코드 수행은 "main.py"로 수행됩니다. 
#     2. "./src/service/workflow_service.py" 이 파일은 참고하지 마세요. 

#     UI 생성을 위한 프로젝트 코드를 작성해주세요
#     결과물은 ./artifacts 에 넣어주세요.
# '''

# user_query = '''
#     "2025년 중국 가전 신제품 출시에 대한 리포트 작성해줘."
# '''

In [5]:
import os
import shutil

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")


import nest_asyncio
nest_asyncio.apply()

remove_artifact_folder()

result = run_agent_workflow(
    user_input=user_query,
    debug=False
)

#빌더에서 플래너 다음에 슈퍼바이져 엣지달기
#슈퍼바이저 에서 코더, 트랙커, 리포터 툴로 만들기 
#코더부터 툴(에이전트 에즈 어 툴)로 만들고 
#트랙커는 새로 만들기


# Print the conversation history
print("\n=== Conversation History ===")
print ("result", result)
for history in result["history"]:

    print ("===")
    print (f'agent: {history["agent"]}')
    print (f'message: {history["message"]}')


INFO [src.workflow] ===== Starting workflow =====

INFO [src.workflow] 
user input: 
    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.
    분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다.
    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.
    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.
    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.
    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.
    최종 리포트는 pdf 형태로 저장해 주세요.


INFO [src.graph.nodes] ===== Coordinator talking...... =====

INFO [src.utils.strands_sdk_utils] COORDINATOR - Prompt Cache Disabled


'./artifacts/' 폴더가 존재하지 않습니다.
handoff_to_planner: I'll need to consult our planning system for this request.

이 요청은 아마존 판매 데이터의 복잡한 분석과 리포트 생성이 필요한 작업이므로, 플래너 시스템의 도움이 필요합니다. 플래너가 데이터 분석, 시각화, PDF 리포트 생성 등의 상세한 계획을 수립하고 실행할 수 있도록 하겠습니다.


INFO [src.graph.nodes] ===== Coordinator updated shared global state with 5 keys =====

INFO [src.graph.nodes] ===== Coordinator completed task =====

INFO [src.graph.nodes] ===== Planner generating plan =====

INFO [src.graph.nodes] 
Planner received task:
[ { 'text': 'Original Task: \n'
            '    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.\n'
            '    분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다.\n'
            '    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.\n'
            '    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.\n'
            '    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.\n'
            '    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.\n'
            '    최종 리포트는 pdf 형태로 저장해 주세요.\n'},
  {'text': '\nInputs from previous nodes:'},
  {'text': '\nFrom coordinator:'},
  { 'text': "  - Agent: handoff_to_planner: I'll need to consult our planning system for this "
            'request.\n'
            '\n'
            '이 요청은 아마존 판매 데이터의 복잡한 분

Let me understand the user's request fully. The user wants to:
1. Analyze Amazon sales data from the file "./data/Amazon_Sale_Report.csv"
2. Extract marketing insights through various analysis techniques
3. Explore data attributes, sales trends, variable relationships, etc.
4. Create a detailed report with visualizations and charts
5. Save the final report as a PDF

This requires a comprehensive plan involving:
- Data loading and exploratory data analysis
- Data visualization and statistical analysis
- Trend analysis and insight extraction
- Report creation with visualizations
- PDF export

Let me create a detailed plan with agent assignments:

1. The Coder will be responsible for all data loading, analysis, and visualization steps
2. The Reporter will create the final PDF report with insights

Since the user has specified the file location, I'll make sure to include that information in the plan.

The plan will need to be structured in a way that the Coder handles all technical aspects


INFO [src.graph.nodes] ===== Planner analyzing and creating execution plan =====

INFO [src.graph.nodes] ===== Planner completed plan generation =====

INFO [src.graph.nodes] ===== Supervisor evaluating next action =====

INFO [src.graph.nodes] 
Supervisor received task:
[ { 'text': 'Original Task: \n'
            '    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.\n'
            '    분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다.\n'
            '    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.\n'
            '    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.\n'
            '    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.\n'
            '    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.\n'
            '    최종 리포트는 pdf 형태로 저장해 주세요.\n'},
  {'text': '\nInputs from previous nodes:'},
  {'text': '\nFrom planner:'},
  { 'text': '  - Agent: # Plan\n'
            '\n'
            '## thought\n'
            'The user wants to analyze Amazon sales data from a CSV file to extract ma

Let me analyze the user's request and the provided plan to determine which worker is best suited to handle it next.

The plan has two main sections:
1. Comprehensive Data Analysis of Amazon Sales Data (Coder tasks)
2. Create Comprehensive Marketing Insights Report (Reporter tasks)

Since we're just starting this project and no "clues" were provided indicating any prior progress, we need to begin from the first task in the plan. The first task is "Load the Amazon sales data from './data/Amazon_Sale_Report.csv'", which is assigned to the Coder.

Looking at the full plan, the logical sequence is to first complete all the data analysis tasks with the Coder, and only then move to the Reporter for creating the final report based on the analysis results.

Therefore, I should assign the first task to the coder_agent to start loading and analyzing the Amazon sales data.# Amazon Sales Data Analysis Project Kickoff

Looking at the provided plan, we need to begin with data analysis before we can c

ERROR:strands.multiagent.graph:node_id=<supervisor>, error=<Expecting value: line 1 column 1 (char 0)> | node failed
ERROR:strands.multiagent.graph:graph execution failed
Traceback (most recent call last):
  File "/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/setup/.venv/lib/python3.12/site-packages/strands/multiagent/graph.py", line 309, in invoke_async
    await self._execute_graph()
  File "/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/setup/.venv/lib/python3.12/site-packages/strands/multiagent/graph.py", line 349, in _execute_graph
    await task
  File "/home/ubuntu/.local/share/uv/python/cpython-3.12.11-linux-aarch64-gnu/lib/python3.12/asyncio/futures.py", line 289, in __await__
    yield self  # This tells Task to wait for completion.
    ^^^^^^^^^^
  File "/home/ubuntu/.local/share/uv/python/cpython-3.12.11-linux-aarch64-gnu/lib/python3.12/asyncio/tasks.py", line 385, in __wakeup
    future.result()
  File "/home/ubu

JSONDecodeError: Expecting value: line 1 column 1 (char 0)